In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --no-cache-dir --force-reinstall
import os
os.kill(os.getpid(), 9)  # Isso reinicia o kernel do Colab


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 148.9 MB/s eta 0:00:00


In [ ]:
!pip install rdkit-pypi
!pip install lazypredict scikit-learn

# Limpeza de dados

In [2]:
import pandas as pd

df = pd.read_csv('/content/Dataset Both.csv')
df.shape

(8195, 5)

In [3]:
from rdkit import Chem

def smiles_valido(smiles):
    """
    Verifica se um SMILES é válido.

    Parâmetros:
        smiles (str): String do SMILES.

    Retorna:
        bool: True se o SMILES é válido, False caso contrário.
    """
    if not isinstance(smiles, str) or smiles.strip() == "":
        return False
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False


In [4]:
df['SMILES_valido'] = df['SMILES'].apply(smiles_valido)
df['SMILES_valido'].value_counts()

[17:06:34] WARNING: not removing hydrogen atom without neighbors
[17:06:34] WARNING: not removing hydrogen atom without neighbors
[17:06:35] WARNING: not removing hydrogen atom without neighbors
[17:06:35] WARNING: not removing hydrogen atom without neighbors
[17:06:35] WARNING: not removing hydrogen atom without neighbors
[17:06:35] WARNING: not removing hydrogen atom without neighbors


,count
SMILES_valido,
True,8195


In [5]:
df.drop(columns=['Chemical', 'Identificador', 'SMILES_valido'], inplace=True)
df.shape

(8195, 3)

In [6]:
df.drop_duplicates(inplace=True)
df.head()
df.shape

(7865, 3)

In [7]:
df['Results'] = df['Results'].str.lower()
df['Type'] = df['Type'].str.lower()

In [8]:
df.drop_duplicates(inplace=True)
df.shape

(6717, 3)

In [9]:
# Mapear os valores para 1 (positivos) e 0 (negativos)
mapping = {"positive": 1, "negative": 0, "ambiguous": None, "inconclusive": None}
df["Results"] = df["Results"].map(mapping)

# Remover valores nulos
df = df.dropna()

In [10]:
df.shape

(6717, 3)

In [11]:
# --- Passo 1 ---
# Para cada SMILES, identificar quais possuem mais de um tipo
smiles_mult_type = df.groupby("SMILES")["Type"].nunique()
smiles_mult_type = smiles_mult_type[smiles_mult_type > 1].index

# Para os SMILES que possuem mais de um 'Type', manter somente as linhas onde 'Type' é 'in vivo'
df_filtrado = df[~(df["SMILES"].isin(smiles_mult_type) & (df["Type"] != "in vivo"))]

# Remover duplicatas, se existirem
df_filtrado = df_filtrado.drop_duplicates()

In [12]:
df_filtrado.shape

(5290, 3)

In [13]:
# --- Passo 2 ---
# Agora, identificar quais SMILES (no df filtrado) possuem mais de um 'Results'
smiles_mult_result = df_filtrado.groupby("SMILES")["Results"].nunique()
smiles_mult_result = smiles_mult_result[smiles_mult_result > 1].index

# Remover os SMILES que possuem mais de um resultado
df_final = df_filtrado[~df_filtrado["SMILES"].isin(smiles_mult_result)]

In [14]:
df_final['Results'].value_counts()

,count
Results,
1.0,1987
0.0,1899


In [15]:
df_final = df_final.drop(columns='Type')

# Estrutura de alerta

In [17]:
from rdkit import Chem
from tqdm import tqdm
from rdkit.Chem import Descriptors,AllChem
from rdkit import RDLogger
import re

RDLogger.DisableLog('rdApp.*')

# Função auxiliar para neutralizar SMILES com cargas
def neutralizar_smiles(smiles):
    """
    Remove cargas formais de átomos representados como [Na+], [Fe+3], [Cl-], etc., convertendo para [Na], [Fe], [Cl], etc.
    """
    if pd.isna(smiles):
        return ""
    return re.sub(r'\[([A-Z][a-z]?)[+-]?[0-9]*\]', r'[\1]', smiles)

# Função para calcular descritores moleculares
def calcular_descritores(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}
    return {desc[0]: desc[1](mol) for desc in Descriptors.descList}

# Função principal com controle de neutralização
def verificar_subestruturas_e_descritores(
    df, df_estruturas,
    smiles_col='SMILES',
    estrutura_smiles_col='SMILES',
    neutralizar=True
):
    """
    Verifica presença de subestruturas e calcula descritores moleculares.

    Parâmetros:
        df: DataFrame com compostos.
        df_estruturas: DataFrame com subestruturas.
        smiles_col: nome da coluna de SMILES no df.
        estrutura_smiles_col: nome da coluna de SMILES no df_estruturas.
        neutralizar: se True, remove carga dos SMILES antes da comparação.

    Retorna:
        DataFrame com descritores e colunas de presença de subestruturas.
    """

    df_estruturas.columns = df_estruturas.columns.str.strip()

    # Preparar os SMILES das subestruturas
    if neutralizar:
        df_estruturas['SMILES_neutro'] = df_estruturas[estrutura_smiles_col].apply(neutralizar_smiles)
    else:
        df_estruturas['SMILES_neutro'] = df_estruturas[estrutura_smiles_col]

    padroes = {
        smiles: Chem.MolFromSmiles(smiles)
        for smiles in df_estruturas['SMILES_neutro']
        if Chem.MolFromSmiles(smiles) is not None
    }

    subestrutura_resultados = {smiles: [] for smiles in padroes}
    descritores_resultados = {desc[0]: [] for desc in Descriptors.descList}

    # Iterar sobre os SMILES dos compostos
    for smiles in tqdm(df[smiles_col], desc="Processando moléculas", unit="molécula"):
        smiles_proc = neutralizar_smiles(smiles) if neutralizar else smiles
        mol = Chem.MolFromSmiles(smiles_proc)

        for sub_smiles, padrao in padroes.items():
            subestrutura_resultados[sub_smiles].append(int(mol.HasSubstructMatch(padrao)) if mol else 0)

        descritores = calcular_descritores(smiles_proc)
        for desc_nome in descritores_resultados:
            descritores_resultados[desc_nome].append(descritores.get(desc_nome, None))

    df_subs = pd.DataFrame(subestrutura_resultados)
    df_descs = pd.DataFrame(descritores_resultados)
    df_final = pd.concat([df.reset_index(drop=True), df_descs, df_subs], axis=1)

    return df_final

In [19]:
df_estruturas = pd.read_csv('/content/Estruturas de alerta.csv')

In [20]:
# Exemplo de uso
df_processado = verificar_subestruturas_e_descritores(
    df=df_final,
    df_estruturas=df_estruturas,
    smiles_col='SMILES',
    estrutura_smiles_col='SMILES'
)
df_processado.shape

Processando moléculas: 100%|██████████| 3886/3886 [00:52<00:00, 73.39molécula/s]


(3886, 362)

# Classificação

In [22]:
# Carregar os dados
df = df_processado.copy()

# Definir as features (X) e o alvo (y)
X = df.drop(columns=['SMILES', 'Results'])  # Remove colunas não numéricas
y = df['Results']

In [ ]:
# Normalizar os dados (opcional, mas recomendado para alguns modelos)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar o StratifiedKFold (10 folds)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Criar dicionário para armazenar os resultados
results = []

# Iterar sobre os folds
for train_idx, test_idx in cv.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inicializar o LazyClassifier
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

    # Treinar e testar os modelos
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    # Salvar os resultados do fold
    results.append(models)

final_results_vivo = (pd.concat(results).groupby(level=0).agg(['mean', 'std']).sort_values(by=('F1 Score', 'mean'), ascending=False)
)

In [39]:
final_results_vivo

Accuracy      Balanced Accuracy      ROC AUC  \
                                  mean  std              mean  std    mean   
Model                                                                        
ExtraTreesClassifier              0.73 0.00              0.73 0.00    0.73   
RandomForestClassifier            0.72 0.01              0.72 0.01    0.72   
LGBMClassifier                    0.71 0.01              0.71 0.01    0.71   
SVC                               0.71 0.00              0.71 0.00    0.71   
XGBClassifier                     0.71 0.00              0.71 0.00    0.71   
NuSVC                             0.71 0.00              0.71 0.00    0.71   
RidgeClassifier                   0.70 0.01              0.70 0.01    0.70   
RidgeClassifierCV                 0.70 0.01              0.70 0.01    0.70   
LogisticRegression                0.70 0.01              0.70 0.01    0.70   
LinearDiscriminantAnalysis        0.70 0.01              0.70 0.01    0.70   
LinearSVC                         0.70 0.01              0.70 0.01    0.70   
BaggingClassifier                 0.69 0.00              0.69 0.00    0.69   
CalibratedClassifierCV            0.69 0.01              0.69 0.01    0.69   
KNeighborsClassifier              0.67 0.01              0.68 0.01    0.68   
ExtraTreeClassifier               0.66 0.00              0.66 0.00    0.66   
LabelSpreading                    0.66 0.00              0.66 0.00    0.66   
LabelPropagation                  0.66 0.00              0.66 0.00    0.66   
DecisionTreeClassifier            0.65 0.01              0.65 0.01    0.65   
AdaBoostClassifier                0.64 0.01              0.64 0.01    0.64   
PassiveAggressiveClassifier       0.64 0.01              0.64 0.01    0.64   
SGDClassifier                     0.63 0.03              0.63 0.03    0.63   
Perceptron                        0.63 0.04              0.63 0.04    0.63   
BernoulliNB                       0.63 0.02              0.63 0.02    0.63   
NearestCentroid                   0.60 0.02              0.60 0.02    0.60   
GaussianNB                        0.59 0.02              0.59 0.02    0.59   
QuadraticDiscriminantAnalysis     0.60 0.01              0.59 0.01    0.59   
DummyClassifier                   0.51 0.00              0.50 0.00    0.50   

                                   F1 Score      Time Taken       
                               std     mean  std       mean  std  
Model                                                             
ExtraTreesClassifier          0.00     0.73 0.00       1.37 0.29  
RandomForestClassifier        0.01     0.72 0.01       1.89 0.32  
LGBMClassifier                0.01     0.71 0.01       1.31 0.01  
SVC                           0.00     0.71 0.00       1.89 0.03  
XGBClassifier                 0.00     0.71 0.00       2.08 1.06  
NuSVC                         0.00     0.71 0.00       1.93 0.07  
RidgeClassifier               0.01     0.70 0.01       0.19 0.13  
RidgeClassifierCV             0.01     0.70 0.01       0.37 0.00  
LogisticRegression            0.01     0.70 0.01       0.23 0.01  
LinearDiscriminantAnalysis    0.01     0.70 0.00       0.31 0.01  
LinearSVC                     0.01     0.70 0.01      17.91 0.59  
BaggingClassifier             0.00     0.69 0.00       2.16 0.04  
CalibratedClassifierCV        0.01     0.69 0.01      71.64 4.94  
KNeighborsClassifier          0.01     0.67 0.01       0.20 0.06  
ExtraTreeClassifier           0.00     0.66 0.00       0.08 0.02  
LabelSpreading                0.00     0.65 0.01       0.77 0.00  
LabelPropagation              0.00     0.65 0.00       0.70 0.18  
DecisionTreeClassifier        0.01     0.65 0.01       0.46 0.09  
AdaBoostClassifier            0.01     0.64 0.01       1.95 0.01  
PassiveAggressiveClassifier   0.01     0.64 0.01       0.14 0.02  
SGDClassifier                 0.03     0.63 0.03       0.29 0.07  
Perceptron                    0.04     0.63 0.04       0.17 0.07  
BernoulliNB   

In [30]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer
import pandas as pd

# Validação cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Métricas que queremos avaliar
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

# Dicionário com modelos, pipelines e grids
modelos = {
    "XGBoost": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6],
            "model__learning_rate": [0.01, 0.1]
        }
    },
    "RandomForest": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", RandomForestClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "ExtraTrees": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", ExtraTreesClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "NuSVC": {
        "pipeline": Pipeline([
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler()),
            ("model", NuSVC(probability=True))
        ]),
        "param_grid": {
            "model__nu": [0.25, 0.5, 0.75],
            "model__kernel": ['rbf', 'poly']
        }
    }
}

# Executar GridSearch para cada modelo e armazenar os resultados
resultados_finais = []

for nome_modelo, config in modelos.items():
    print(f"\n🔍 Treinando modelo: {nome_modelo}")

    grid = GridSearchCV(
        estimator=config["pipeline"],
        param_grid=config["param_grid"],
        scoring=scoring,
        refit="f1",
        cv=cv,
        verbose=1,
        n_jobs=-1,
        return_train_score=True
    )

    grid.fit(X, y)

    # Resultados em DataFrame
    df_resultado = pd.DataFrame(grid.cv_results_)
    df_resultado['modelo'] = nome_modelo
    resultados_finais.append(df_resultado)

    # Mostrar melhores parâmetros do modelo atual
    print(f"✅ Melhor F1 ({nome_modelo}): {grid.best_score_:.4f}")
    print(f"Melhores parâmetros: {grid.best_params_}")

# Juntar todos os resultados
df_comparacao = pd.concat(resultados_finais, ignore_index=True)

# Filtrar e ordenar os principais resultados por F1
colunas_mostrar = ['modelo', 'mean_test_accuracy', 'mean_test_precision',
                   'mean_test_recall', 'mean_test_f1', 'mean_test_roc_auc', 'params']

df_resultados_finais = df_comparacao[colunas_mostrar].sort_values(
    by="mean_test_f1", ascending=False)


🔍 Treinando modelo: XGBoost
Fitting 5 folds for each of 8 candidates, totalling 40 fits
✅ Melhor F1 (XGBoost): 0.7271
Melhores parâmetros: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 100}

🔍 Treinando modelo: RandomForest
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Melhor F1 (RandomForest): 0.6984
Melhores parâmetros: {'model__max_depth': 6, 'model__n_estimators': 100}

🔍 Treinando modelo: ExtraTrees
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Melhor F1 (ExtraTrees): 0.6484
Melhores parâmetros: {'model__max_depth': 6, 'model__n_estimators': 100}

🔍 Treinando modelo: NuSVC
Fitting 5 folds for each of 6 candidates, totalling 30 fits
✅ Melhor F1 (NuSVC): 0.7316
Melhores parâmetros: {'model__kernel': 'rbf', 'model__nu': 0.5}


# Melhor modelo

In [32]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, StratifiedKFold
import numpy as np

# Pipeline com scaler + modelo
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ExtraTreesClassifier(random_state=42))
])

# Estratégia de validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Avaliar o modelo usando cross_validate com múltiplas métricas
resultados = cross_validate(
    pipeline,
    X, y,
    cv=cv,
    scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
    return_train_score=True
)

# Mostrar métricas médias
print("🔍 Resultados médios na validação cruzada:")
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    mean = resultados[f'test_{metric}'].mean()
    std = resultados[f'test_{metric}'].std()
    print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")


🔍 Resultados médios na validação cruzada:
accuracy  : 0.7316 ± 0.0199
precision : 0.7349 ± 0.0223
recall    : 0.7438 ± 0.0209
f1        : 0.7392 ± 0.0188
roc_auc   : 0.8027 ± 0.0242


# Conclusão

Os resultados obtidos por meio da validação cruzada demonstram um desempenho consistente e satisfatório do modelo avaliado. A acurácia média de 73,16% indica uma boa capacidade global de classificação, enquanto os valores de precisão (73,49%) e revocação (74,38%) apontam para um desempenho equilibrado entre a correta identificação das classes positivas e a minimização de falsos positivos.

O F1-score médio de 0,7392 reforça essa harmonia entre precisão e revocação, sendo particularmente relevante em contextos onde o balanceamento entre essas métricas é crítico. Além disso, o AUC-ROC médio de 0,8027 revela uma adequada capacidade discriminativa do modelo, evidenciando sua eficácia na diferenciação entre as classes.

Os baixos desvios padrão observados entre os folds da validação cruzada sugerem estabilidade e robustez no comportamento do modelo, o que é desejável para aplicações em ambientes com variabilidade nos dados. Tais resultados indicam que o modelo está bem ajustado ao problema proposto, apresentando potencial para ser utilizado em contextos práticos ou como base para iterações adicionais no pipeline de modelagem.

# Conclusion

The results obtained through cross-validation demonstrate a consistent and satisfactory performance of the evaluated model. The mean accuracy of 73.16% indicates good overall classification capability, while the precision (73.49%) and recall (74.38%) values reflect a balanced performance in correctly identifying positive instances and minimizing false positives.

The mean F1-score of 0.7392 further supports this balance between precision and recall, which is particularly important in scenarios where both metrics are critical. Additionally, the mean AUC-ROC of 0.8027 highlights the model’s ability to effectively discriminate between classes.

The low standard deviations observed across cross-validation folds suggest that the model exhibits stable and robust behavior, which is desirable for applications involving data variability. These results indicate that the model is well-suited to the problem at hand and may serve as a reliable solution in practical settings or as a foundation for further optimization within the modeling pipeline.